# **Explanation of Multi-Armed Bandit Algorithms**
The most popular Multi-Armed Bandit algorithms are presented here, including Thompson Sampling, which is the algorithm we use to optimize campaigns on the teaser-level.

In [1]:
from simulator import (HumanAgent, EpsilonGreedyAgent, ThompsonSamplingAgent,
                       UCBAgent, NoOptimizationAgent, OptimizationGame)
from numpy.random import default_rng
rng = default_rng()
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
n_teasers = 5
dist_alpha = 2.0
dist_beta = 1000.0
true_ctrs = rng.beta(a=dist_alpha, b=dist_beta, size=n_teasers)

## **$\epsilon$-greedy**
The idea of the $\epsilon$-greedy algorithm is simple: With probability $1 - \epsilon$, play the teaser which has currently the highest empirical $ctr$ (exploitation step). With probability $\epsilon$, randomly choose one of the other teasers (exploration). The value of $\epsilon$ needs to be set by the user. High $\epsilon$ values drive exploration, low $\epsilon$ values drive exploitation. Reasonable values for $\epsilon$ are $1\% \lesssim \epsilon \lesssim 10\%$. Let's play an optimization game with an $\epsilon$-greedy agent only and look more closely what it's doing by setting the option `verbose=True`. The empirical $ctr$'s are visualized in chart of blue bars. The teaser that will be activated until the next configuration update is highlighted in orange color. We use $\epsilon = 20\%$ to observe an exploration step more often:

In [3]:
e_greedy_agent = EpsilonGreedyAgent(epsilon=.2, n_teasers=n_teasers, n_views_update_data=10000,
                                    n_views_update_configuration=3000, verbose=True)
game = OptimizationGame([e_greedy_agent], true_ctrs=true_ctrs, n_views_total=1000000)
game.start()

New data was imported!
New data was imported!
Known data after daily import:


,active,n_clicks,n_views,empirical ctr
teaser 1,False,5,1953,0.26%
teaser 2,True,14,2040,0.69%
teaser 3,False,2,2040,0.10%
teaser 4,False,4,2041,0.20%
teaser 5,False,3,1926,0.16%


Chosen teaser =  teaser 2


KeyboardInterrupt: Interrupted by user

$\epsilon$-greedy is a very simple algorithm that shows surprisingly good results. However, in the limit of infinite views, the best teaser is not played with a probability of $\epsilon$ even though it is known with certainty. A schedule to decrease $\epsilon$ over time can be applied, but a badly chosen schedule can completely mess up the optimization.

## **Thompson Sampling**
Thompson Sampling uses Bayes' law to compute distributions over the $ctr$'s given a certain amount of clicks and views ([for details, check the Insights documentation here](https://content-garden.gitlab.io/analytics-server/theory.html#thompson-sampling)). It then plays a teaser with the same probability as the probability of this teaser to be the best one ("probability matching decision strategy"). It achieves this by drawing random numbers from the inferred probability distributions over the $ctr$'s and then playing the teaser corresponding to the highest random number that was drawn.

Assume that, after recording some clicks and views for, say, 3 different teasers, we know that the first one has $10\%$, the second one $20\%$ and the third one $70\%$ probability to be the best teaser. Then Thompson Sampling will play the first one $10\%$, the second one $20\%$ and the third one $70\%$ of the time.

After some more views, our knowledge of the true teaser $ctr$'s increases and it may be that according to our knowledge, the first teaser is with $2\%$, the second with $5\%$ and the third with $93\%$ the best teaser. Then the first one is played $2\%$, the second $5\%$ and the third one $93\%$ of the time.

It may as well be that actually the second teaser is better than the third and there was only bad luck for the second one in the first couple of views. Then our knowlege about the best teaser may refine to, e.g., $4\%$ for the first, $60\%$ for the second and $36\%$ for the third and the Thompson Sampling agent will play the teasers accordingly.

In the limit of infinite views, Thompson Sampling will always find the best teaser and play it almost $100\%$ of the time in future steps.

A nice feature is that this algorithm does not require any fine tuning of parameters as is necessary in, e.g., the $\epsilon$-greedy method. 

Let's set up a Thompson Sampling agent and set `verbose=True` to see some visualizations. Observe that whenever data is imported ("daily import"), the probability distributions over the teaser $ctr$'s change. Whenever the configuration is updated, random numbers from these distributions are drawn and visualized as dots on the $x$-axis of the same color as the probability distribution of the corresponding teaser $ctr$. Only the teaser with the largest drawn random number is activated, indicated by an encircled dot and a bigger line width. 

In [ ]:
ts_agent = ThompsonSamplingAgent(n_views_update_data=10000, n_views_update_configuration=3000, n_teasers=n_teasers,
                                 prior_alpha=dist_alpha, prior_beta=dist_beta, verbose=True)
game = OptimizationGame([ts_agent], true_ctrs=true_ctrs, n_views_total=1000000)
game.start()

## **Upper Confidence Bounds**
The UCB algorithm is pretty similar to Thompson Sampling, but instead of drawing random numbers from the inferred probability distributions over the teaser $ctr$'s, upper confidence bounds are computed and the teaser with largest value is selected. A common upper confidence bound is the $95\%$ quantile (i.e., the $ctr$ value where to $95\%$ probability, the true $ctr$ is smaller, and to $5\%$ it's bigger).

We specify a UCB agent, draw the probability distributions over the $ctr$'s at every configuration update, and indicate the upper confidence bounds ($95\%$ quantiles) with dots on the $x$-axis.

In [ ]:
ucb_agent = UCBAgent(n_views_update_data=10000, n_views_update_configuration=3000, n_teasers=n_teasers,
                                 prior_alpha=dist_alpha, prior_beta=dist_beta, ucb_quantile=0.95, verbose=True)
game = OptimizationGame([ucb_agent], true_ctrs=true_ctrs, n_views_total=1000000)
game.start()

New data was imported!
New data was imported!
Known data after daily import:


,active,n_clicks,n_views,empirical ctr
teaser 1,False,4,1922,0.21%
teaser 2,True,17,2080,0.82%
teaser 3,False,1,2018,0.05%
teaser 4,False,8,1938,0.41%
teaser 5,False,1,2042,0.05%


Chosen teaser =  teaser 2


The UCB algorithm shows similar or even better performance than Thompson Sampling. The issue in our context is that it will always play the same teaser until fresh data is imported, i.e., only a single active teaser for a whole day.

# **Summary**
The three most common Multi-Armed Bandit algorithms ($\epsilon$-greedy, UCB, and Thompson Sampling) were presented. The Bayesian methods UCB and Thompson Sampling generally show the best performance. However, applying UCB would in our case mean that only a single teaser is active on a campaign/placement pair for a whole day since configurations do not change until there is fresh data.

Therefore, in our context, Thompson Sampling seems to be most promising.

To better align Thompson Sampling with the restriction of only limited configuration updates, strategies with more than a single active teaser can be developed.